<a href="https://colab.research.google.com/github/Ireneyou33/Model-Comparison-and-Forecasting-on-COVID-19/blob/master/functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !echo %cd%

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout
import datetime as dt
from tensorflow.python.keras.layers import Dense, LSTM
from tensorflow.python.keras import Sequential
from keras.models import load_model
from keras.models import model_from_json
from datetime import datetime, timedelta

Using TensorFlow backend.


In [ ]:
df_conf=pd.read_csv(r"C:\Users\hp\COVID-19\Confirmed.csv")
df_death=pd.read_csv(r"C:\Users\hp\COVID-19\Deaths.csv")
df_recover=pd.read_csv(r"C:\Users\hp\COVID-19\Recovered.csv")

In [ ]:
df_conf.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,27878,28424,28833,29157,29481,29640,30175,30451,30616,30967
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1838,1891,1962,1995,2047,2114,2192,2269,2330,2402
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,172,176,183,186,189,197,212,212,259,267


In [ ]:
# Get the world class dataframe
def worldclass_dataframe(dataframe):
    date=dataframe.columns[4:]
    for i in range(len(date)):
        date.values[i]=pd.to_datetime(date.values[i], format="%m/%d/%y")
    world=dataframe.sum(axis=0, skipna=True)
    worldclass=world.iloc[2:,]
    arr=np.array(worldclass)
    new_df=pd.DataFrame([arr],columns=date)
    return new_df

In [ ]:
def sum_regioncase(dataframe, region, region_level="county"):
    # region: county, region or outside China
    if region_level=="county":
        df_return=dataframe[dataframe["Country/Region"]==region].iloc[:,4:]
    elif region_level=="province":
        df_return=dataframe[dataframe["Province/State"]==region].iloc[:,4:]
    else:
        df_return=dataframe[dataframe["Country/Region"]!=region].iloc[:,4:]
    reg=df_return.sum(axis=0, skipna=True)
    arr=np.array(reg)
    date=df_return.columns
    for i in range(len(date)):
        date.values[i]=pd.to_datetime(date.values[i], format="%m/%d/%y")
    new_df=pd.DataFrame([arr],columns=date)
    return new_df, arr, date

In [ ]:
 def train_test(data, split_index):
    trainset=data.iloc[:, :split_index]
    testset=data.iloc[:, split_index:]
    return trainset, testset

In [ ]:
def WindowSlider(data, w=1, r=1):
  #w: window size
  #r: response size

    cols=len(list(data))
    l=cols-(w+r)+1
    df = pd.DataFrame(np.zeros(shape=(l, w+w+2*r)))

    for m in range(l):
        slices=np.array([])
        delta_t=[]
        num_confirm=[]

        for i in range(w):
            i+=m
            delta=data.columns[i]-start_date
            delta=delta.floor('d').days
            delta_t.append(delta)
            num_confirm.append(data.iloc[0,i])

        y_delta=data.columns[m+w+r-1]-start_date
        y_delta=y_delta.floor('d').days
        y_delta=np.array(y_delta).reshape(1,)
        y=data.iloc[0,m+w+r-1]
        y=np.array(y).reshape(1,)

        slices=np.concatenate((slices, delta_t, num_confirm))
        slices=np.concatenate((slices, y_delta, y))

        df.iloc[m,:]=slices
    return df

In [ ]:
def get_colnames(data, w=1, r=1):
    delta_names=[]
    x_names=[]
    for i in range(w):
        delta_names.append('∆t' + ('(%d)' % i))
        x_names.append('x'+ ('%d' % i))
    col_names=delta_names+x_names
    l=w+r-1
    col_names.append('∆t' + ('(%d)' % l))
    col_names.append('y')
    return col_names

### Daily Bar Plot

In [ ]:
def daily_bar_plot(dataframe, title, c=None):
    daily_df = dataframe.copy()
    length=dataframe.shape[1]
    for i in range(length-1):
        daily_df.iloc[0,i+1]=dataframe.iloc[0,i+1]-dataframe.iloc[0,i]
    daily_df.T.plot.bar(title=title, figsize=(25,10), legend=False, color=c)
    return daily_df

### Barh Plot

In [ ]:
def barh_plot(dataframe, title, country=True, c=None):
    df=dataframe.iloc[:,[1,-1]]
    if country:
        sum_region = df.groupby(['Country/Region']).sum()
    else:
        print("Not country level")
    sum_region.columns=["date"]
    sum_region=sum_region.sort_values("date",ascending=False)
    new_df=sum_region.head(n=20)
    ax = new_df.plot.barh(color=c)
    ax.set_title(title)
    return new_df

### Pie Plot

In [ ]:
def pie_plot(dataframe, stage, title, inplace=False):
    # stage = "Confirmed", "Deaths", "Recovered"
    dataframe2=dataframe.copy()
    dataframe2.reset_index(inplace=inplace)
    dataframe2.columns=["Country",stage]
    region=np.array(dataframe2.index.values)
    values=np.array(dataframe2.iloc[:,0])
    if stage == "Confirmed":
        fig = px.pie(dataframe2, names= 'Country', values=stage,title=title, color_discrete_sequence=px.colors.sequential.Blues)
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.show()
    elif stage == "Deaths":
        fig = px.pie(dataframe2, names= 'Country', values=stage,title=title, color_discrete_sequence=px.colors.sequential.RdBu)
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.show()
    elif stage == "Recovered":
        fig = px.pie(dataframe2, names= 'Country', values=stage,title=title, color_discrete_sequence=px.colors.sequential.Greens)
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.show()
    else:
        print("Oooops Undefined Stage")

## NeuralNets

In [ ]:
def get_traintest(dataframe, idx, w=1, r=1, wfv=False):
    if wfv==True:
        trainset=dataframe.iloc[:, :idx]
        testset=dataframe.iloc[:, idx:idx+10] # fixed test size = 10
    else:
        trainset, testset = train_test(dataframe, split_index=idx)
    if w == 1:
        train = WindowSlider(trainset, w=w, r=1)
        X_train = train[1]
        Y_train = train[3]
        test = WindowSlider(testset, w=w, r=1)
        X_test = test[1]
        Y_test = test[3]
    elif w == 3:
        train = WindowSlider(trainset, w=w, r=1)
        X_train = train[[3,4,5]]
        Y_train = train[[7]]
        test = WindowSlider(testset, w=w, r=1)
        X_test = test[[3,4,5]]
        Y_test = test[[7]]
    elif w == 5:
        train = WindowSlider(trainset, w=w, r=1)
        X_train = train[[5,6,7,8,9]]
        Y_train = train[[11]]
        test = WindowSlider(testset, w=w, r=1)
        X_test = test[[5,6,7,8,9]]
        Y_test = test[[11]]
    elif w == 7:
        train = WindowSlider(trainset, w=w, r=1)
        X_train = train[[7,8,9,10,11,12,13]]
        Y_train = train[[15]]
        test = WindowSlider(testset, w=w, r=1)
        X_test = test[[7,8,9,10,11,12,13]]
        Y_test = test[[15]]
    return X_train, Y_train, X_test, Y_test

## Average R0 dataframe

In [ ]:
def ave_R0(conf_dataframe, recov_dataframe, death_dataframe, ave_days=7):
  # Parameters estimation
    slope=[]
    gamma=[]
    mu=[]
    for i in range(0, conf_dataframe.shape[1]-1):
        s=np.log(conf_dataframe.iloc[0,i+1])-np.log(conf_dataframe.iloc[0,i])
        g=(recov_dataframe.iloc[0,i+1]-recov_dataframe.iloc[0,i])/conf_dataframe.iloc[0,i]
        m=(death_dataframe.iloc[0,i+1]-death_dataframe.iloc[0,i])/conf_dataframe.iloc[0,i]
        slope.append(s)
        gamma.append(g)
        mu.append(m)
    beta=np.array(slope)+np.array(gamma)+np.array(mu)
    # Average R0
    R0=[]
    for i in range(len(beta)):
        if gamma[i]+mu[i] != 0:
            R0.append(beta[i]/(gamma[i]+mu[i]))
        else:
            R0.append(0)
    ave_R0=[]
    ave_R0+=[sum(R0[i:i+ave_days])/ave_days for i in range(len(R0)-ave_days+1)]
    # Region average R0 dataframe
    arr=np.array(ave_R0)
    date=conf_dataframe.columns[:-ave_days]
    for i in range(len(date)):
        date.values[i]=pd.to_datetime(date.values[i], format="%m/%d/%y")
    df_region_aveR0=pd.DataFrame([arr],columns=date)
    return df_region_aveR0

## Use LSTM to model R0

In [ ]:
def get_traintestdata(dataframe, w=1, r=1, lockdown_date=1):
    if w == 3:
        start_date = pd.to_datetime(dataframe.columns[lockdown_date], format="%m/%d/%y")
        training = WindowSlider(dataframe.iloc[:,lockdown_date:], w)
        x_train = training[[3,4,5]]
        y_train = training[[7]]
    elif w ==1:
        start_date = pd.to_datetime(dataframe.columns[lockdown_date], format="%m/%d/%y")
        training = WindowSlider(dataframe.iloc[:,lockdown_date:], w)
        x_train = training[[1]]
        y_train = training[[3]]
    return x_train, y_train

In [ ]:
def lstm_r0_model(dataframe, w=1, r=1, lockdown_date=1):
    x_train, y_train = get_traintestdata(dataframe)
    x = np.array(x_train)
    x = x.astype(int)
    X = x.reshape((x.shape[0], x.shape[1], r))

    # define model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(w, r)))

    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X, y_train, epochs=200, verbose=0)
    if w == 3:
        model.save('HubeiR0_w3.h5')
    elif w == 1:
        model.save('HubeiR0_w1.h5')

In [ ]:
def R0_forecast(df, model, forecast_days=10, w=3, r=1):
    df_test = df.copy()
    for i in range(forecast_days):
        a = df_test.columns[-1] + timedelta(days=1)
        df_test[a] = np.nan
        df_test[a].iloc[0] = df_test.iloc[1,-2]
        testing = WindowSlider(df_test, w)
        if w == 3:
            x_test = testing[[3,4,5]]
            y_test = testing[[7]]
        elif w == 1:
            x_test = testing[[1]]
            y_test = testing[[3]]
        x_test = np.array(x_test)
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], r))
        y_pred = model.predict(x_test, verbose=0)
        df_test.iloc[-1,-1]=y_pred[-1]
    return df_test

Walk Forward Validation

In [ ]:
def walk_forward_vali(dataframe, w=1, idx=30):
    train_RMSE=[]
    test_RMSE=[]
    train_size=[]
    while idx < dataframe.shape[1]-10:
        X_tr, Y_tr, X_te, Y_te = get_traintest(dataframe, idx, w=w, r=1, wfv=True)
        model = Sequential()
        model.add(Dense(8, input_dim=w, activation='relu'))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_tr, Y_tr, epochs=200, batch_size=2, verbose=0)
        trainScore = model.evaluate(X_tr, Y_tr, verbose=0)
        train_RMSE.append(math.sqrt(trainScore))
        testScore = model.evaluate(X_te, Y_te, verbose=0)
        test_RMSE.append(math.sqrt(testScore))
        train_size.append(idx)
        idx+=10 # Each time increase training size by 10
    return train_RMSE, test_RMSE, train_size

In [ ]:
def walk_forward_vali_lstm(dataframe, w=1, idx=30):
    train_RMSE=[]
    test_RMSE=[]
    train_size=[]
    while idx < dataframe.shape[1]-10:
        X_tr, Y_tr, X_te, Y_te = get_traintest(dataframe, idx, w=w, r=1, wfv=True)
        if w ==1:
            X_tr = reshape_func(X_tr, r=1)
            X_te = reshape_func(X_te, r=1)
        else:
            X_tr = reshape_func(X_tr,shape=w, r=1)
            X_te = reshape_func(X_te, shape=w, r=1)
        
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(w, r)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_tr, Y_tr, epochs=200, batch_size=2, verbose=0)
        trainScore = model.evaluate(X_tr, Y_tr, verbose=0)
        train_RMSE.append(math.sqrt(trainScore))
        testScore = model.evaluate(X_te, Y_te, verbose=0)
        test_RMSE.append(math.sqrt(testScore))
        train_size.append(idx)
        idx+=10 # Each time increase training size by 10
    return train_RMSE, test_RMSE, train_size